In [1]:
import pandas as pd
import sqlalchemy as alch
import mysql.connector
from mysql.connector import connect
from getpass import getpass

#### See SQL code here

USE sakila;

CREATE TABLE rental_may AS
SELECT rental_id, customer_id
FROM rental
WHERE MONTH(rental_date) = 5;

SELECT COUNT(*)
FROM rental_may;

CREATE TABLE rental_june AS
SELECT rental_id, customer_id
FROM rental
WHERE MONTH(rental_date) = 6;

SELECT COUNT(*)
FROM rental_june;

In [2]:
db = connect(
host = "127.0.0.1",
user = 'root',
passwd = getpass('Enter password:'),
database = 'sakila') 

Enter password:········


In [3]:
cursor = db.cursor()

In [4]:
num_rentals_may = "SELECT customer_id, COUNT(rental_id) as num_rentals FROM rental_may GROUP BY customer_id"

In [5]:
cursor.execute(num_rentals_may)

In [6]:
df_may = pd.DataFrame(cursor.fetchall())
df_may

,0,1
0,130,2
1,459,1
2,408,3
3,333,1
4,222,5
...,...,...
515,191,2
516,351,1
517,10,1
518,136,1


In [8]:
num_rentals_june = "SELECT customer_id, COUNT(rental_id) as num_rentals FROM rental_june GROUP BY customer_id"

In [9]:
cursor.execute(num_rentals_june)

In [10]:
df_june = pd.DataFrame(cursor.fetchall())
df_june

,0,1
0,416,5
1,516,6
2,239,5
3,285,3
4,310,6
...,...,...
585,412,1
586,335,1
587,226,2
588,22,1


In [14]:
df_may = df_may.rename(columns={0: 'customer_id', 1: 'rentals_may'})
df_may

,customer_id,rentals_may
0,130,2
1,459,1
2,408,3
3,333,1
4,222,5
...,...,...
515,191,2
516,351,1
517,10,1
518,136,1


In [15]:
df_june = df_june.rename(columns={0: 'customer_id', 1: 'rentals_june'})
df_june

,customer_id,rentals_june
0,416,5
1,516,6
2,239,5
3,285,3
4,310,6
...,...,...
585,412,1
586,335,1
587,226,2
588,22,1


In [16]:
df_merged = pd.merge(df_may, df_june, on='customer_id', how='outer')

print(df_merged)

     customer_id  rentals_may  rentals_june
0            130          2.0           6.0
1            459          1.0           7.0
2            408          3.0           3.0
3            333          1.0           4.0
4            222          5.0           2.0
..           ...          ...           ...
593          598          NaN           1.0
594          487          NaN           1.0
595          555          NaN           1.0
596          335          NaN           1.0
597          226          NaN           2.0

[598 rows x 3 columns]


In [18]:
df_merged.fillna(0, inplace=True)
print(df_merged)

     customer_id  rentals_may  rentals_june
0            130          2.0           6.0
1            459          1.0           7.0
2            408          3.0           3.0
3            333          1.0           4.0
4            222          5.0           2.0
..           ...          ...           ...
593          598          0.0           1.0
594          487          0.0           1.0
595          555          0.0           1.0
596          335          0.0           1.0
597          226          0.0           2.0

[598 rows x 3 columns]


In [20]:
df_merged['rental_comparison'] = df_merged.apply(lambda row: 'Less rentals in June' if row['rentals_may'] > row['rentals_june'] 
                                                             else 'More rentals in June' if row['rentals_may'] < row['rentals_june'] 
                                                             else 'Equal', axis=1)
df_merged

,customer_id,rentals_may,rentals_june,rental_comparison
0,130,2.0,6.0,More rentals in June
1,459,1.0,7.0,More rentals in June
2,408,3.0,3.0,Equal
3,333,1.0,4.0,More rentals in June
4,222,5.0,2.0,Less rentals in June
...,...,...,...,...
593,598,0.0,1.0,More rentals in June
594,487,0.0,1.0,More rentals in June
595,555,0.0,1.0,More rentals in June
596,335,0.0,1.0,More rentals in June
